Identify the model -> YOLO

Image classification Object Detection Instance segmentation

video, images, audio, text

Object detection

Model - yolo 11
decide what object to identify - Umbrella (done)

Gather our dataset (done)

Labe;/Annotate

Train

Validation

Inference

In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 55.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 123.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 89.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 61.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 74.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12

In [ ]:
!nvidia-smi

Sat Aug 16 16:10:18 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   51C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
# Unzip images to a custom data folder
!unzip -q /content/data.zip -d /content/custom_data

In [ ]:
import glob
import shutil # Import shutil for moving files
from pathlib import Path # Import Path


script_content = """
# Split between train and val folders

from pathlib import Path
import random
import os
import sys
import shutil
import argparse
import glob # Added glob import

# Define and parse user input arguments

parser = argparse.ArgumentParser()
parser.add_argument('--datapath', help='Path to data folder containing image and annotation files',
                    required=True)
parser.add_argument('--imagepath', help='Path to data folder containing image and annotation files',
                    required=True)
parser.add_argument('--labelpath', help='Path to data folder containing image and annotation files',
                    required=True)
parser.add_argument('--train_pct', help='Ratio of images to go to train folder; \\
                    the rest go to validation folder (example: ".8")',
                    default=.8)

args = parser.parse_args()

data_path = args.datapath
image_path = args.imagepath
label_path = args.labelpath
train_percent = float(args.train_pct)

# Check for valid entries
if not os.path.isdir(data_path):
   print('Directory specified by --datapath not found. Verify the path is correct (and uses double back slashes if on Windows) and try again.')
   sys.exit(0)
if train_percent < .01 or train_percent > 0.99:
   print('Invalid entry for train_pct. Please enter a number between .01 and .99.')
   sys.exit(0)
val_percent = 1 - train_percent

# Define paths to image and annotation folders
# Modified to use the data_path directly
input_image_path = image_path
input_label_path = label_path

cwd = os.getcwd()
train_img_path = os.path.join(cwd,'data/train/images')
train_txt_path = os.path.join(cwd,'data/train/labels')
val_img_path = os.path.join(cwd,'data/validation/images')
val_txt_path = os.path.join(cwd,'data/validation/labels')

# Create folders if they don't already exist
for dir_path in [train_img_path, train_txt_path, val_img_path, val_txt_path]:
   if not os.path.exists(dir_path):
      os.makedirs(dir_path)
      print(f'Created folder at {dir_path}.')


# Get list of all images and annotation files directly from datapath
img_file_list = [Path(f) for f in glob.glob(os.path.join(input_image_path, "*.*")) if f.lower().endswith((".jpg", ".jpeg", ".png", ".bmp", ".tiff"))]
txt_file_list = [Path(f) for f in glob.glob(os.path.join(input_label_path, "*.txt"))]

print(f'Number of image files: {len(img_file_list)}')
print(f'Number of annotation files: {len(txt_file_list)}')

# Pair image and label files with the same base name
paired_files = []
for img_path in img_file_list:
    label_filename = img_path.with_suffix('.txt').name
    label_path = Path(input_label_path) / label_filename
    if label_path.exists():
        paired_files.append((img_path, label_path))

# Determine number of files to move to each folder based on paired files
file_num = len(paired_files)
train_num = int(file_num * train_percent)
val_num = file_num - train_num
print('Images moving to train: %d' % train_num)
print('Images moving to validation: %d' % val_num)

# Shuffle the paired files and split into train and validation sets
random.shuffle(paired_files)
train_files = paired_files[:train_num]
val_files = paired_files[train_num:]

# Copy train images and annotations to train folders
for img_path, label_path in train_files:
    image_filename = img_path.name
    annotation_filename = label_path.name
    src_image_path = str(img_path)
    src_annotation_path = str(label_path)
    dest_image_path = os.path.join(train_img_path, image_filename)
    dest_annotation_path = os.path.join(train_txt_path, annotation_filename)
    shutil.copy(src_image_path, dest_image_path)
    shutil.copy(src_annotation_path, dest_annotation_path)
    #print(f'Copied {image_filename} and {annotation_filename} to train')


# Copy validation images and annotations to validation folders
for img_path, label_path in val_files:
    image_filename = img_path.name
    annotation_filename = label_path.name
    src_image_path = str(img_path)
    src_annotation_path = str(label_path)
    dest_image_path = os.path.join(val_img_path, image_filename)
    dest_annotation_path = os.path.join(val_txt_path, annotation_filename)
    shutil.copy(src_image_path, dest_image_path)
    shutil.copy(src_annotation_path, dest_annotation_path)
    #print(f'Copied {image_filename} and {annotation_filename} to validation')

"""


with open('/content/train_val_split.py', 'w') as f:
    f.write(script_content)


# Run the modified script
!python train_val_split.py --datapath="/content/custom_data" --imagepath="/content/custom_data/images" --labelpath="/content/custom_data/labels" --train_pct=0.8 #0.9 --> 90% training rest for test

Directory specified by --datapath not found. Verify the path is correct (and uses double back slashes if on Windows) and try again.


In [ ]:
# Python function to automatically create data.yaml config file
# 1. Reads "classes.txt" file to get list of class names
# 2. Creates data dictionary with correct paths to folders, number of classes, and names of classes
# 3. Writes data in YAML format to data.yaml

import yaml
import os

def create_data_yaml(path_to_classes_txt, path_to_data_yaml):

  # Read class.txt to get class names
  if not os.path.exists(path_to_classes_txt):
    print(f'classes.txt file not found! Please create a classes.txt labelmap and move it to {path_to_classes_txt}')
    return
  with open(path_to_classes_txt, 'r') as f:
    classes = []
    for line in f.readlines():
      if len(line.strip()) == 0: continue
      classes.append(line.strip())
  number_of_classes = len(classes)

  # Create data dictionary
  data = {
      'path': '/content/data',
      'train': 'train/images',
      'val': 'validation/images',
      'nc': number_of_classes,
      'names': classes
  }

  # Write data to YAML file
  with open(path_to_data_yaml, 'w') as f:
    yaml.dump(data, f, sort_keys=False)
  print(f'Created config file at {path_to_data_yaml}')

  return

# Define path to classes.txt and run function
path_to_classes_txt = '/content/custom_data/classes.txt'
path_to_data_yaml = '/content/data.yaml'

create_data_yaml(path_to_classes_txt, path_to_data_yaml)

print('\nFile contents:\n')
!cat /content/data.yaml

classes.txt file not found! Please create a classes.txt labelmap and move it to /content/custom_data/classes.txt

File contents:

cat: /content/data.yaml: No such file or directory


In [ ]:
!yolo detect train data=/content/data.yaml model=yolo11s.pt epochs=100 imgsz=640

In [ ]:
!yolo detect predict model=runs/detect/train/weights/best.pt source=data/validation/images save=True

In [ ]:
import glob
from IPython.display import Image, display
for image_path in glob.glob(f'/content/runs/detect/predict/*.jpg')[:10]:
  display(Image(filename=image_path, height=600, width=900))
  print('\n')

In [ ]:
!mkdir /content/my_model
!cp /content/runs/detect/train/weights/best.pt /content/my_model/my_model.pt
!cp -r /content/runs/detect/train /content/my_model

# Zip
%cd my_model
!zip /content/my_model.zip my_model.pt
!zip -r /content/my_model.zip train
%cd /content

In [ ]:
from ultralytics import YOLO

# Load the model
model = YOLO("/content/my_model/my_model.pt")

In [ ]:
import cv2
from google.colab.patches import cv2_imshow
import numpy as np
from PIL import Image


# Run inference on an image
results = model("/content/chh.jpg", conf=0.3)
print(results)
# Process results
for r in results:
    im_array = r.plot()  # plot a BGR numpy array of predictions
    im = Image.fromarray(im_array[..., ::-1])  # RGB PIL image
    # Convert the PIL Image to a numpy array (OpenCV format)
    im_cv2 = cv2.cvtColor(np.array(im), cv2.COLOR_RGB2BGR)

    # Display the image using cv2_imshow
    cv2_imshow(im_cv2)
    print('\n')

In [ ]:
from ultralytics import YOLO

# Load the model
model = YOLO("/content/my_model/my_model.pt")

In [ ]:
# Run inference on a video
# Replace "your_video.mp4" with the path to your video file
results = model("/content/vv.mp4", conf=0.5, save=True)

In [ ]:
import os
from IPython.display import display, Video

# Define the path to the saved video. Assumes the video is saved in 'runs/detect/predict'
# You might need to adjust the path based on the actual output directory of the previous cell.
output_dir = 'runs/detect/predict3'
# Find the latest created video file in the output directory
video_files = [f for f in os.listdir(output_dir) if f.endswith(('.mp4', '.avi'))]
video_files.sort(key=lambda x: os.path.getmtime(os.path.join(output_dir, x)))

if video_files:
    video_path = os.path.join(output_dir, video_files[-1])
    print(f"Displaying video from: {video_path}")

    # Display the video directly in the notebook
    display(Video(video_path, embed=True))
else:
    print("No video file found in the output directory.")